In [1]:
from lib.tasks import (
    extract_legiscan_datasets, 
    get_legiscan_datasets, 
    prepare_neutral_corpus,
    process_legiscan_datasets,
)
import pandas as pd

WORK_DIR = '../tmp/neutral_corpus'
SUMMARIES = '../tmp/neutral_corpus/summaries.json'

get_legiscan_datasets(WORK_DIR)
extract_legiscan_datasets(WORK_DIR)
process_legiscan_datasets(WORK_DIR, SUMMARIES)

aggregate_frame = pd.read_json('../artifacts/aggregate.json')
states_with_bills = aggregate_frame.state.unique().tolist()

neutral_bills = prepare_neutral_corpus(WORK_DIR, SUMMARIES, 1000, filter_states=states_with_bills)
neutral_bills

['../tmp/neutral_corpus/bills/SC_H3299.html',
 '../tmp/neutral_corpus/bills/TN_SB1274.pdf',
 '../tmp/neutral_corpus/bills/TX_SB311.html',
 '../tmp/neutral_corpus/bills/IA_HF5.html',
 '../tmp/neutral_corpus/bills/NV_AB187.pdf',
 '../tmp/neutral_corpus/bills/IL_SB0722.html',
 '../tmp/neutral_corpus/bills/VA_HB2140.html',
 '../tmp/neutral_corpus/bills/MT_HB725.pdf',
 '../tmp/neutral_corpus/bills/RI_H5049.pdf',
 '../tmp/neutral_corpus/bills/MN_HF505.pdf',
 '../tmp/neutral_corpus/bills/RI_H5426.pdf',
 '../tmp/neutral_corpus/bills/RI_H5754.pdf',
 '../tmp/neutral_corpus/bills/MS_SB2304.html',
 '../tmp/neutral_corpus/bills/OR_HB2668.pdf',
 '../tmp/neutral_corpus/bills/RI_H5969.pdf',
 '../tmp/neutral_corpus/bills/VA_HB1476.html',
 '../tmp/neutral_corpus/bills/IA_HSB57.html',
 '../tmp/neutral_corpus/bills/MD_HB322.pdf',
 '../tmp/neutral_corpus/bills/NM_SM54.pdf',
 '../tmp/neutral_corpus/bills/RI_H5442.pdf',
 '../tmp/neutral_corpus/bills/TX_HR101.html',
 '../tmp/neutral_corpus/bills/TN_HB0420.pdf